Install folium if not already installed
pip install folium

In [ ]:
import folium

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib.patches as mpatches

In [ ]:
def split_coordinates(file_path="parsedPowerPlant/DE_power_plant_coords.csv"):
    df = pd.read_csv(file_path)
    # Split into lantitude and longtitude and round with 4 decimal places
    df[['la', 'lo']] = df['coordinate'].str.split(',', expand=True).astype(float).round(4)
    # print(df.head())
    df.to_csv("DE_power_plant_coords_split_rounded.csv", index=False)

In [ ]:
def read_csv_into_df(file_path="DE_power_plant_coords_split_rounded.csv"):
    df = pd.read_csv(file_path, usecols=["coordinate", "resourceName"])\
       .assign(lat=lambda x: x['coordinate'].str.split(',', expand=True)[0].astype(float),
               lon=lambda x: x['coordinate'].str.split(',', expand=True)[1].astype(float))\
       .rename(columns={"resourceName": "name"})[["lat", "lon", "name"]]
    print(df.head())

    return df

split_coordinates()
read_csv_into_df()

In [ ]:
# --- 1️⃣ Load LAU shapefile ---
lau = gpd.read_file("data/lau2024/LAU_RG_01M_2024_4326.shp")
lau_de = lau[lau["CNTR_CODE"] == "DE"].copy()  # Only Germany

In [ ]:
# Ensure CRS is consistent
lau_de = lau_de.to_crs(epsg=4326)

In [ ]:
# --- 2️⃣ Load power plant coordinates ---
plants = read_csv_into_df()
# plants = pd.DataFrame({
#     "name": ["Mainz KW2", "Weisweiler", "Munich HKW Süd", "BASF Süd_GT1", "Witznau Hydropower"],
#     "lat": [50.027, 50.819, 48.096, 49.51373, 47.6876],
#     "lon": [8.241, 6.288, 11.547, 8.431556, 8.2513]
# })
gdf_plants = gpd.GeoDataFrame(
    plants,
    geometry=gpd.points_from_xy(plants.lon, plants.lat),
    crs="EPSG:4326"
)

In [ ]:
# --- 3️⃣ Manual counting: assign each plant to the LAU polygon ---
lau_de["plant_count"] = 0

In [ ]:
for idx, plant in gdf_plants.iterrows():
    matches = lau_de.contains(plant.geometry)
    lau_de.loc[matches, "plant_count"] += 1

In [ ]:
# Ensure integer type
lau_de["plant_count"] = lau_de["plant_count"].astype(int)

In [ ]:
print(sum(lau_de["plant_count"]))

In [ ]:
# --- 4️⃣ Categorize LAUs ---
def categorize_plants(n):
    if n == 0:
        return "0 plants"
    elif n == 1:
        return "1 plant"
    elif 2 <= n <= 4:
        return "2-4 plants"
    else:
        return ">4 plants"

In [ ]:
lau_de["plant_category"] = lau_de["plant_count"].apply(categorize_plants)

In [ ]:
# --- 5️⃣ Color mapping ---
color_dict = {
    "0 plants": "#FFDAB9",       # light orange
    "1 plant": "#90EE90",        # light green
    "2-4 plants": "#66C2A5",     # medium green
    ">4 plants": "#006400"       # dark green
}

In [ ]:
# --- 6️⃣ Plotting ---
fig, ax = plt.subplots(figsize=(10, 12))

In [ ]:
# Plot LAU polygons colored by category
lau_de.plot(
    ax=ax,
    color=lau_de["plant_category"].map(color_dict),
    edgecolor="saddlebrown",
    linewidth=0.4
)

In [ ]:
# Draw Germany border on top
lau_de.boundary.plot(ax=ax, color="saddlebrown", linewidth=1)

In [ ]:
# Plot power plants
gdf_plants.plot(ax=ax, color="red", markersize=30, zorder=5)

Add labels for plants
for x, y, label in zip(gdf_plants.geometry.x, gdf_plants.geometry.y, gdf_plants["name"]):
    ax.text(x + 0.05, y, label, fontsize=9, zorder=6, color="black")

In [ ]:
# Add custom legend
legend_handles = [mpatches.Patch(color=c, label=l) for l, c in color_dict.items()]
ax.legend(handles=legend_handles, fontsize=8,title="Number of Plants per LAU", loc="lower right")

In [ ]:
# Final touches
ax.set_title("Germany — LAU Regions Categorized by Power Plant Count", fontsize=16)
ax.set_axis_off()
plt.savefig("DE_power_plant.pdf")
plt.show()